# BibRec: Training Random Forest Model

### Evaluation API: Predict top rated books for given User with imported model
Input for the model is a list of top rated books along with the user information

In [26]:
from bibrec.server.Utils import *

books, users, ratings = get_normalized_data(books_path='data/BX-Books.csv',
                                            users_path='data/BX-Users.csv',
                                            ratings_path='data/BX-Book-Ratings.csv')
books = books.drop(["isbn"], axis=1)
ratings = ratings.drop(["isbn"], axis=1)

With NaN values 34.862889904962536
used mean values 34.86974551937334


/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:41: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)
/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explicit_filtered_ratings['isbn'] = explicit_filtered_ratings['isbn'].apply(lambda x: x.upper())
/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path, sep=";", encoding="latin-1")


With NaN values 34.862889904962536
used mean values 34.8606971947035


/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:41: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)
/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  explicit_filtered_ratings['isbn'] = explicit_filtered_ratings['isbn'].apply(lambda x: x.upper())


In [27]:
# limit data to top-rated books only
top_ratings = ratings.groupby('isbn13').user_id.count().sort_values(ascending=False)
top_ratings = top_ratings[:100]
top_ratings = top_ratings.reset_index()
# user_id => nr of ratings
top_ratings = top_ratings.drop('user_id', axis=1)
top_ratings

,isbn13
0,9780971880108
1,9780316666343
2,9780385504201
3,9780060928339
4,9780312195519
...,...
95,9780142000205
96,9780804107532
97,9780345351524
98,9780449212608


In [28]:
explicit_ratings = ratings[ratings.book_rating != 0]
top_ratings = top_ratings.merge(explicit_ratings, on='isbn13', how='inner')
top_ratings

,isbn13,user_id,book_rating
0,9780971880108,277042,2
1,9780971880108,278176,9
2,9780971880108,899,2
3,9780971880108,1435,5
4,9780971880108,1486,5
...,...,...,...
18866,9780140280098,265784,5
18867,9780140280098,268657,9
18868,9780140280098,269831,7
18869,9780140280098,272573,8


In [29]:
filtered_books = books[books.isbn13.isin(top_ratings.isbn13)]
filtered_books

,book_title,book_author,year_of_publication,publisher,image_url_s,image_url_m,image_url_l,isbn13,rating_mean,rating_count,normalized_year_of_publication
18,The Testament,John Grisham,1999,other,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,http://images.amazon.com/images/P/0440234743.0...,9780440234746,7.694118,170.0,6
26,Wild Animus,Rich Shapero,2004,other,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,9780971880108,4.390706,581.0,1
28,Timeline,MICHAEL CRICHTON,2000,ballantine_books,http://images.amazon.com/images/P/0345417623.0...,http://images.amazon.com/images/P/0345417623.0...,http://images.amazon.com/images/P/0345417623.0...,9780345417626,7.851282,195.0,5
37,To Kill a Mockingbird,Harper Lee,1988,other,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,9780446310789,8.943925,214.0,17
52,The Street Lawyer,JOHN GRISHAM,1999,other,http://images.amazon.com/images/P/0440225701.0...,http://images.amazon.com/images/P/0440225701.0...,http://images.amazon.com/images/P/0440225701.0...,9780440225706,7.624060,133.0,6
...,...,...,...,...,...,...,...,...,...,...,...
152675,The Perfect Storm : A True Story of Men Agains...,Sebastian Junger,1998,other,http://images.amazon.com/images/P/006101351X.0...,http://images.amazon.com/images/P/006101351X.0...,http://images.amazon.com/images/P/006101351X.0...,9780061013515,7.386555,119.0,7
188832,A Painted House,John Grisham,2001,other,http://images.amazon.com/images/P/044023722X.0...,http://images.amazon.com/images/P/044023722X.0...,http://images.amazon.com/images/P/044023722X.0...,9780440237228,7.340426,282.0,4
192189,The Bridges of Madison County,Robert James Waller,1992,warner_books,http://images.amazon.com/images/P/044651652X.0...,http://images.amazon.com/images/P/044651652X.0...,http://images.amazon.com/images/P/044651652X.0...,9780446516525,7.506579,152.0,13
203629,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,9780439358064,9.038647,207.0,2


In [30]:
encoded_books = hot_encode_books(filtered_books)
df_books = encoded_books.filter(regex="isbn13|normalized_year_of_publication|publisher_", axis=1)
df_books

,isbn13,normalized_year_of_publication,publisher_harlequin,publisher_silhouette,publisher_pocket,publisher_ballantine_books,publisher_bantam_books,publisher_scholastic,publisher_simon_&amp;_schuster,publisher_penguin_books,...,publisher_harpercollins,publisher_fawcett_books,publisher_signet_book,publisher_random_house_inc,publisher_st_martins_pr,publisher_st._martin's_press,publisher_tor_books,publisher_harpercollins_publishers,publisher_zebra_books,publisher_other
18,9780440234746,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26,9780971880108,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28,9780345417626,5,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37,9780446310789,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52,9780440225706,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152675,9780061013515,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
188832,9780440237228,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
192189,9780446516525,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203629,9780439358064,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df_user = pd.DataFrame([{
    'age': 20,
    'city': None,
    'state': None,
    'country': "something"
}])
df_user = normalize_country(df_user, users.country)
df_user = normalize_state(df_user)
df_user = hot_encode_users(df_user)
df_user = df_user.filter(regex="age|country_|state_", axis=1)
df_user

,age,country_usa,country_canada,country_united_kingdom,country_germany,country_spain,country_australia,country_italy,country_france,country_portugal,...,state_ohio,state_michigan,state_oregon,state_virginia,state_massachusetts,state_missouri,state_nordrhein-westfalen,state_north_carolina,state_,state_other
0,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
df_input = df_books.assign(**df_user.iloc[0])
df_input

,isbn13,normalized_year_of_publication,publisher_harlequin,publisher_silhouette,publisher_pocket,publisher_ballantine_books,publisher_bantam_books,publisher_scholastic,publisher_simon_&amp;_schuster,publisher_penguin_books,...,state_ohio,state_michigan,state_oregon,state_virginia,state_massachusetts,state_missouri,state_nordrhein-westfalen,state_north_carolina,state_,state_other
18,9780440234746,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26,9780971880108,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28,9780345417626,5,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
37,9780446310789,17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52,9780440225706,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152675,9780061013515,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
188832,9780440237228,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
192189,9780446516525,13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
203629,9780439358064,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [33]:
from os.path import exists
import pickle

# Load the model from the file
model_file = "rf5.pkl"

if not exists(model_file):
    exit("No model found")

print("Loading file:", model_file)
with open(model_file, "rb") as file:
    rfc = pickle.load(file)


Loading file: rf5.pkl


FileNotFoundError: [Errno 2] No such file or directory: 'rf5.pkl'

In [ ]:
rfc_predict = rfc.predict(df_input)
pd.DataFrame(rfc_predict)

In [ ]:
import seaborn as sns
sns.countplot(x = rfc_predict)

In [ ]:
predictions = df_books.filter(regex="isbn13", axis=1)
predictions = predictions.reset_index()
predicted_ratings = pd.DataFrame(rfc_predict, columns=["predicted_book_rating"])
predictions = predictions.join(predicted_ratings)
predictions = predictions.join(predicted_ratings)
predictions

In [ ]:
predictions = predictions.sort_values("predicted_book_rating", na_position="first", ascending=False)
predictions

In [ ]:
predictions.merge(books, on="isbn13", how="left")